In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

import talib
from talib.abstract import *
from talib import MA_Type

from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest

/mnt/c/Users/Anshul/Documents/experimental/env/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [102]:
from lib.pivots import getHHIndex, getHLIndex, getLLIndex, getLHIndex
#Create a strategy
class TrendFollow(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    order = 1
    
    sl = 0
    
    def init(self):
        # Precompute the two moving averages
        pass
    
    def next(self):
        hh_val = 0
        hl_val = 0
        lh_val = 0
        ll_val = 0
        d = self.data.df.index[-1].to_pydatetime()
        if (d.hour == 9 and d.minute==15) or (d.hour == 15 and d.minute>15):
            hh_val = 0
            hl_val = 0
            lh_val = 0
            ll_val = 0
            self.sl = 0
            return
        elif d.hour == 9 and d.minute<20:
            #No decision making in the first 5 minutes
            return
        elif d.hour==15 and d.minute==15:
            if self.position:
                #print('Close position')
                self.position.close()
                return
        hh = getHHIndex(self.data.Close, order=self.order)
        if hh is not None and len(hh)>1:
            hh = hh[-1]-self.order
            hh_val = self.data.Close[hh]
        else:
            hh = -1
        hl = getHLIndex(self.data.Close, order=self.order)
        if hl is not None and len(hl)>1:
            hl = hl[-1]-self.order
            hl_val = self.data.Close[hl]
        else:
            hl = -1
        ll = getLLIndex(self.data.Close, order=self.order)
        if ll is not None and len(ll)>1:
            ll = ll[-1]-self.order
            ll_val = self.data.Close[ll]
        else:
            ll = -1
        lh = getLHIndex(self.data.Close, order=self.order)
        if lh is not None and len(lh)>1:
            lh = lh[-1]-self.order
            lh_val = self.data.Close[lh]
        else:
            lh = -1
        
        #print(f'D: {self.data.df.index[-1]} HH:{hh} HL:{hl} LH:{lh} LL:{ll}')
        if hh>ll and hl>ll:
            #Colud be uptrend
            if hh > hl:
                #uptrend confirm
                if not self.position  and self.data.Close[-1] > hl_val:
                    #print('Long')
                    #print(self.data.df.tail(1))
                    self.sl = hl_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.buy(sl = self.sl)
                elif self.position.is_short  and self.data.Close[-1] > hl_val:
                    #print('Long')
                    #print(self.data.df.tail(1))
                    self.sl = hl_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.buy(sl = self.sl)
                else:
                    if self.sl != hl_val:
                        self.sl = hl_val
                        #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')
                    #Else, update stop loss
            else:
                if hl > hh and self.data.Close[-1] > hh_val:
                    #uptrend pending confirm, but candle just closed above previous high
                    if not self.position and self.data.Close[-1] > hl_val:
                        #print('Long')
                        #print(self.data.df.tail(1))
                        self.sl = hl_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.buy(sl = self.sl)
                    elif self.position.is_short and self.data.Close[-1] > hl_val:
                        #print('Long')
                        #print(self.data.df.tail(1))
                        self.sl = hl_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.buy(sl = self.sl)
                    else:
                        if self.sl != hl_val:
                            self.sl = hl_val
                            #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')
                        #Else, update stop loss
        if lh > hh and lh > hl:
            #downtrend possible
            if ll > lh:
                #downtrend confirm
                if not self.position and self.data.Close[-1] < lh_val:
                    #print('Short')
                    #print(self.data.df.tail(1))
                    self.sl = lh_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.sell(sl=self.sl)
                elif self.position.is_long  and self.data.Close[-1] < lh_val:
                    #print('Short')
                    #print(self.data.df.tail(1))
                    self.sl = lh_val
                    #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                    self.sell(sl=self.sl)
                else:
                    #Else, update stoploss
                    if self.position and self.sl != lh_val:
                        self.sl = lh_val
                        #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')
            else:
                if lh > ll and self.data.Close[-1] < ll_val:
                    #downtrend confirm pending, but we're crossing over the lower low
                    if not self.position and self.data.Close[-1] < lh_val:
                        #print('Short')
                        #print(self.data.df.tail(1))
                        self.sl = lh_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.sell(sl=self.sl)
                    elif self.position.is_long  and self.data.Close[-1] < lh_val:
                        #print('Short')
                        #print(self.data.df.tail(1))
                        self.sl = lh_val
                        #print(f'D: {self.data.df.index[-1]} O:{self.data.Open[-1]} H:{self.data.High[-1]} L:{self.data.Low[-1]} C:{self.data.Close[-1]} SL:{self.sl}')
                        self.sell(sl=self.sl)
                    else:
                        #Else, update stoploss
                        if self.position and self.sl != lh_val:
                            self.sl = lh_val
                            #print(f'D: {self.data.df.index[-1]} SL: {self.sl}')

In [103]:
df = pd.read_csv('./NIFTY50_1m.csv', index_col=0)
for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)

#df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
#df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":'Volume'}, inplace=True)

df = df[~df.index.duplicated(keep='first')]
#Optionally, filter out by date range
#print(df.head(10))
filter = True
if filter:
    start_date = '2015-09-01 09:15:00'
    end_date = '2015-09-30 03:30:00'
    df = df.loc[start_date:end_date]

df.tail()

,Open,High,Low,Close,Volume
date,,,,,
2015-09-29 15:25:00,7828.25,7828.95,7825.5,7826.15,0
2015-09-29 15:26:00,7826.05,7827.40,7825.6,7825.60,0
2015-09-29 15:27:00,7825.60,7827.95,7824.7,7827.50,0
2015-09-29 15:28:00,7827.90,7828.85,7826.3,7827.60,0
2015-09-29 15:29:00,7827.55,7831.85,7826.1,7829.70,0


In [104]:
bt = Backtest(df, TrendFollow, cash=100000, commission=0/100000, trade_on_close=False, exclusive_orders = True)
stats = bt.run()
print(stats)

bt.plot()

Start                     2015-09-01 09:15:00
End                       2015-09-29 15:29:00
Duration                     28 days 06:14:00
Exposure Time [%]                   73.159747
Equity Final [$]                     101691.8
Equity Peak [$]                     103267.75
Return [%]                             1.6918
Buy & Hold Return [%]                -0.81894
Return (Ann.) [%]                     6.43166
Volatility (Ann.) [%]               13.596101
Sharpe Ratio                         0.473052
Sortino Ratio                        0.827974
Calmar Ratio                         1.697137
Max. Drawdown [%]                   -3.789712
Avg. Drawdown [%]                   -0.306736
Max. Drawdown Duration       22 days 05:35:00
Avg. Drawdown Duration        0 days 21:11:00
# Trades                                  373
Win Rate [%]                        21.179625
Best Trade [%]                       1.803175
Worst Trade [%]                     -0.366009
Avg. Trade [%]                    

Row(id='6928', ...)